<a href="https://colab.research.google.com/github/tulasiram58827/Information-Extraction-From-Documents/blob/main/SROIE_Candidate_Generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homepage: https://rrc.cvc.uab.es/?ch=13&com=downloads. 

In [1]:
!gdown --id 10r9y17wg8Elo-3Zi61xA_8QDaKix8giN -O data.tar.xz

Downloading...
From: https://drive.google.com/uc?id=10r9y17wg8Elo-3Zi61xA_8QDaKix8giN
To: /content/data.tar.xz
247MB [00:02, 83.0MB/s]


In [2]:
!tar -xf data.tar.xz

In [1]:
import cv2
import glob
import json
import imutils
import numpy as np

# https://stackoverflow.com/a/51855662
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import matplotlib.pyplot as plt

from tqdm import tqdm
from imutils import paths
from google.colab.patches import cv2_imshow

In [2]:
images = sorted(list(paths.list_images("/content/data/img")))
len(images)

626

In [3]:
csvs = sorted(glob.glob("/content/data/box/*.csv"))
len(csvs)

626

In [4]:
jsons = sorted(glob.glob("/content/data/key/*.json"))
len(jsons)

626

In [5]:
csvs[0]

'/content/data/box/000.csv'

In [6]:
sample_csv = pd.read_csv(csvs[0], names=["x1_1", "y1_1", "x2_1", "y2_1", 
                                         "x3_1", "y3_1", "x4_1", "y4_1", "transcript"])
sample_csv.head()

,x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1,transcript
0,72,25,326,25,326,64,72,64,TAN WOON YANN
1,50,82,440,82,440,121,50,121,BOOK TA .K(TAMAN DAYA) SDN BND
2,205,121,285,121,285,139,205,139,789417-W
3,110,144,383,144,383,163,110,163,NO.53 55
4,192,169,299,169,299,187,192,187,TAMAN DAYA


In [7]:
!pip install -q dateparser
import dateparser

In [8]:
def parse_dates(row):
    return dateparser.parse(row, settings={'STRICT_PARSING': True, 
                                           'REQUIRE_PARTS': ['day', 'month', 'year']})

sample_csv["date_candidate"] = sample_csv["transcript"].apply(parse_dates)
sample_csv["date_candidate"] = pd.to_datetime(sample_csv['date_candidate'], errors = 'coerce').dt.date
sample_csv.sample(5)

,x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1,transcript,date_candidate
13,191,460,298,460,298,476,191,476,CASH BILL,NaT
34,402,748,441,748,441,763,402,763,10.00,NaT
6,217,216,275,216,275,233,217,233,JOHOR.,NaT
38,190,864,309,864,309,880,190,880,EXCHANGEABLE,NaT
23,77,598,113,598,113,613,77,613,1 PC,NaT


In [9]:
sample_csv["total_candidate"] = pd.to_numeric(sample_csv["transcript"], errors="coerce")
sample_csv.sample(5)

,x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1,transcript,date_candidate,total_candidate
20,420,529,443,529,443,547,420,547,RM,NaT,nan
5,162,193,334,193,334,211,162,211,81100 JOHOR BAHRU,NaT,nan
43,412,639,442,639,442,654,412,654,9.00,NaT,9.000
23,77,598,113,598,113,613,77,613,1 PC,NaT,nan
26,275,598,309,598,309,612,275,612,0.00,NaT,0.000


In [10]:
sample_csv_filtered = sample_csv[(~sample_csv['date_candidate'].isnull()) | (~sample_csv['total_candidate'].isnull())]
sample_csv_filtered

,x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1,transcript,date_candidate,total_candidate
9,165,372,342,372,342,389,165,389,25/12/2018 8:13:39 PM,2018-12-25,nan
21,27,570,137,570,137,583,27,583,9556939040116,NaT,9556939040116.000
25,202,597,245,597,245,612,202,612,9.000,NaT,9.000
26,275,598,309,598,309,612,275,612,0.00,NaT,0.000
27,411,596,443,596,443,613,411,613,9.00,NaT,9.000
30,408,669,443,669,443,684,408,684,0.00,NaT,0.000
32,401,703,443,703,443,719,401,719,9.00,NaT,9.000
34,402,748,441,748,441,763,402,763,10.00,NaT,10.000
36,412,772,443,772,443,786,412,786,1.00,NaT,1.000
43,412,639,442,639,442,654,412,654,9.00,NaT,9.000
